## Introduction

LaVague QA is a command-line tool that turns Gherkin test specifications into ready-to-use Pytest code for web applications. Built on the LaVague open-source framework, it automates the creation and maintenance of automated tests.


### How it works

1. Write test scenarios in natural language using Gherkin.
2. Run LaVague QA to automatically generate the corresponding Pytest code.
3. Execute tests or regenerate them as your website evolves.

# Installation

## Start by installing
- `lavague-qa`
- `pytest-bdd`

In [ ]:
!pip install lavague-qa pytest-bdd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of llama-index-llms-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-program-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 88.6 MB

### Get the `OPENAI_API_KEY` from the environment.

Add your key to the secrets tab if you're running in colab

In [ ]:
import os

# Check if running in Google Colab
try:
    from google.colab import userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
else:
    os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# 1. Define test scenario
We fetch a `.feature` file from the LaVague repository and display it

In [ ]:
!wget https://raw.githubusercontent.com/lavague-ai/LaVague/main/lavague-qa/features/demo_wikipedia.feature

--2024-08-05 07:11:30--  https://raw.githubusercontent.com/lavague-ai/LaVague/rework-qa-automation-example/lavague-qa/features/demo_wikipedia.feature
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 426 [text/plain]
Saving to: ‘demo_wikipedia.feature’

demo_wikipedia.feat 100%[===================>]     426  --.-KB/s    in 0s      

2024-08-05 07:11:30 (8.91 MB/s) - ‘demo_wikipedia.feature’ saved [426/426]



In [ ]:
!cat demo_wikipedia.feature

Feature: Wikipedia Login

  Scenario: User logs in successfully
    Given the user is on the Wikipedia homepage
    When the user navigates to the login page
    And the user enters Lavague-test in the username field
    And the user enters lavaguetest123 in the password field
    And the user clicks on login under the username and password field
    Then the login is successful and the user is redirected to the main page


# 2. Run LaVague QA

LaVague QA is a CLI tool, as such, we run it with `!` in notebooks.

We will run in `--headless` mode since colab cannot display a browser window. LaVague will run all steps defined in your tests and record actions and selectors. If you're running locally, you can remove this flag.

In [ ]:
!lavague-qa --headless --url https://en.wikipedia.org/ --feature ./demo_wikipedia.feature

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/lavague/core/__init__.py:21: UserWarning: Telemetry is turned on. To turn off telemetry, set your LAVAGUE_TELEMETRY to 'NONE'
  warnings.warn(warning_message, UserWarning)
Ready to generate tests on https://en.wikipedia.org/ for Feature: Wikipedia Login

  Scenario: User logs in successfully
    Given the user is on the Wikipedia homepage
    When the user navigates to the login page
    And the user enters Lavague-test in the username field
    And the user enters lavaguetest123 in the password field
    And the user clicks on login under the use

We now display the pytest file generated.

In [ ]:
!cat generated_tests/demo_wikipedia.py

## Run the following cell if in colab, otherwise you can skip this cell.

Since colab only allows headless browsers, we need to inject a `--headless` argument to our pytest fixture before we start the brower driver.

This cell opens the file, replaces the code and writes the file back to disk.

In [ ]:
FILE_PATH = "generated_tests/demo_wikipedia.py"

REPLACEMENT = (
"""from selenium.webdriver.chrome.options import Options
    from selenium import webdriver
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)"""
)


import os

# read file
with open(FILE_PATH, 'r') as file:
    content = file.read()

modified_content = content.replace("driver = webdriver.Chrome()", REPLACEMENT)

# write file
with open(FILE_PATH, 'w') as file:
    file.write(modified_content)

print(f"Updated file: {FILE_PATH}")

Let's look at the file after injecting a new fixture.

In [ ]:
!cat generated_tests/demo_wikipedia.py

# Run the tests

We can now run tests using `pytest`

In [ ]:
!pytest generated_tests/demo_wikipedia.py